<a href="https://colab.research.google.com/github/Julian27R/Trabajos/blob/main/Ejercicios_simulaci%C3%B3n_SyS_nov_16-Laplace/Punto_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**EJERCICIO**

4. Fije los valores de los párametros del sistema rlc paralelo, con condiciones iniciales cero, para simular la respuesta subamortiguada, sobreamortiguada, de amortiguamiento critico, oscilatoria e inestable, ante entrada escalón.


#**SOLUCIÓN**
##Circuito RLC Paralelo:
* Salida ante una entrada Escalon.

In [1]:
import sympy as sym
import matplotlib.pyplot as plt
import sympy.physics.control as control

sym.init_printing()

s = sym.symbols('s', complex=True)
t, R, L, C, = sym.symbols('t R L C ', positive=True)
#Entrada de interés en Laplace
X = 1/s # Ajustar según tipo de entrada
H=control.TransferFunction(1,(s**2 + s/(R*C) + 1/(L*C))*L*C,s) #Funcion de transferencia con la API de control
#Hlc = H/(1+H) #para lazo cerrado

Y = H.num * X /H.den#función de transferencia del circuito multiplicada por X(s)
#Función de salida
Y

          1           
──────────────────────
      ⎛ 2    s     1 ⎞
C⋅L⋅s⋅⎜s  + ─── + ───⎟
      ⎝     C⋅R   C⋅L⎠

Para simular la respuesta subamortiguada, sobreamortiguada, de amortiguamiento critico, oscilatoria e inestable, ante entrada escalón. Es necesario encontrar la forma canónica SLIT de segundo orden.

In [19]:
s = sym.symbols('s', complex=True)
t,R, L, C = sym.symbols('t R L C', positive=True, real=True)

#Entrada de interés en Laplace
X = 1/s # X(s) = 1/s resp escalón
K = 1 #ganancia estática
wn = sym.sqrt(1/(C*L)) #frecuencia natural
xi = sym.sqrt(C*L) / (2*R*C) #factor amortiguamiento
H = K*(wn**2/(s**2 + 2*xi*wn*s + wn**2)) #función de transferencia en lazo cerrado slit 2 orden
#Hlc = H/(1+H) #para lazo cerrado
H

         1          
────────────────────
    ⎛ 2    s     1 ⎞
C⋅L⋅⎜s  + ─── + ───⎟
    ⎝     C⋅R   C⋅L⎠

* Respuesta Subamortiguada

In [46]:
#Le damos valores a R, C, L para mostrar la respuesta Subamortiguada
sym.init_printing()
R = R.subs({R : 1})
C = C.subs({C : 0.4})
L = L.subs({L : 0.5})
xi = sym.sqrt(C*L) / (2*R*C) #factor amortiguamiento
xi #'\tCuando 0 < xi < 1; la respuesta del SLIT es subamortiguada'

0.559016994374947

In [47]:
#Le damos valores a R, C, L para mostrar la respuesta sobreamortiguada
R = R.subs({R :1})
C = C.subs({C : 1})
L = L.subs({L :10})
xi = sym.sqrt(C*L) / (2*R*C) #factor amortiguamiento
xi #('\tCuando xi > 1; la respuesta del SLIT es sobreamortiguada')

√10
───
 2 

In [48]:
#Le damos valores a R, C, L para mostrar la respuesta de amortiguamineto critico
R = R.subs({R :1})
C = C.subs({C : 1})
L = L.subs({L :4})
xi = sym.sqrt(C*L) / (2*R*C) #factor amortiguamiento
xi #('\tCuando xi = 1; la respuesta del SLIT es de amortiguamiento critico')

1

In [49]:
#Le damos valores a R, C, L para mostrar la respuesta oscilatoria
R = R.subs({R :1})
C = C.subs({C : 1})
L = L.subs({L : 0})
xi = sym.sqrt(C*L) / (2*R*C) #factor amortiguamiento
xi #('\tCuando xi = 0; la respuesta del SLIT es oscilatoria')

0

In [52]:
#Le damos valores a R, C, L para mostrar la respuesta inestable
R = R.subs({R :-1})
C = C.subs({C : 1})
L = L.subs({L : 1})
xi = sym.sqrt(C*L) / (2*R*C) #factor amortiguamiento
xi #('\tCuando xi < 0; la respuesta del SLIT es inestable')

-1/2